
Screw actually taking the right float epilon. Do numerical analysis style kind of. Higham

Approximation rewriting - knuckled rewriting combinators. Run to possibly just get reasomable bound
Approximation completion
Approximation egraph
Approximation Calc - make Calc environment, but track epsilon
Approximation congruence closure - Lipshitz constants or interval propagators

approximate definition. `approx_add(x,y) :={eps} x + y`

triangle inequality. any metric
|t - s| + |s - u| <= |t - u|


(x e+ y) e+ z ={4eps} x e+ (y e+ z) 

t ={eps} s
if defined as
|t - s| <= eps

Intuitively it makes sense.
Search becomes shittier.
Shortest path.
But reasonable spanning trees can also be ok.


There is something "higher D" like a 2 category or proof relevance that we may need to do rewriting on our symbolic errors also.
It's something like conditional rewriting.


ultrametrics?






In [ ]:
from kdrag.all import *

approx_add = smt.Function("~add", smt.RealSort(),smt.RealSort(), smt.RealSort())

x,y,z, eps = smt.Reals("x y z eps")
add_defn = kd.axiom(smt.ForAll([x,y], 
    smt.Abs(approx_add(x,y) - (x + y)) <= 0.001 , patterns=[approx_add(x,y)]) 
    )
    # * smt.Abs(x + y)
add_comm = kd.axiom(smt.ForAll([x,y], approx_add(x,y) == approx_add(y,x), patterns=[approx_add(x,y)]))

approx = kd.define("approx", [x,y,eps], smt.Abs(x - y) <= eps ) #* smt.Abs(x)

kd.prove(approx(approx_add(x, approx_add(y,z)),
  approx_add(approx_add(x,y), z),
 0.003), unfold=2, by=[add_defn], timeout=5000)


       approx(x, y, eps) ==
       (If(x - y > 0, x - y, -(x - y)) <=
        eps*If(x > 0, x, -x))) to ForAll([x, y, eps],
       approx(x, y, eps) ==
       (If(x - y > 0, x - y, -(x - y)) <= eps))


TimeoutError: Timeout. Maybe you have given `prove` too many or not enough lemmas?

In [35]:
@kd.PTheorem(approx(approx_add(x, approx_add(y,z)),
  approx_add(approx_add(x,y), z),
 0.01))
def approx_add_assoc(l):
    l.unfold(approx)
    l.simp()
    l.have(approx(approx_add(x,y), x + y, 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(x,approx_add(y,z)), x + approx_add(y,z), 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(y,z), y + z, 0.001), by=[add_defn, approx.defn])
    l.have(approx(approx_add(x,y) + z, x + y + z, 0.001), by=[add_defn, approx.defn])
    l.unfold(approx, at=0)
    l.unfold(approx, at=1)
    l.unfold(approx, at=2)
    

Next Goal:
 [If(~add(x, y) - (x + y) > 0,
   ~add(x, y) - (x + y),
   -(~add(x, y) - (x + y))) <=
1/1000,
 If(~add(x, ~add(y, z)) - (x + ~add(y, z)) > 0,
   ~add(x, ~add(y, z)) - (x + ~add(y, z)),
   -(~add(x, ~add(y, z)) - (x + ~add(y, z)))) <=
1/1000,
 If(~add(y, z) - (y + z) > 0,
   ~add(y, z) - (y + z),
   -(~add(y, z) - (y + z))) <=
1/1000,
 approx(~add(x, y) + z, x + y + z, 1/1000)]
?|= If(~add(x, ~add(y, z)) + -1*~add(~add(x, y), z) <= 0,
   -1*~add(x, ~add(y, z)) + ~add(~add(x, y), z),
   ~add(x, ~add(y, z)) + -1*~add(~add(x, y), z)) <=
1/100
